In [17]:
import pandas as pd
import time

from datetime import datetime, timedelta
from google_play_scraper import Sort, reviews

In [ ]:
# get all reviews ranging from today to first instance of day before yesterday
# drop columns 
# rename columns
# filter rows - remove reviews from today and reviews from day before yesterday

# insert to database

In [21]:
def get_extraction_is_complete(data):
    today = datetime.today()
    today = pd.Timestamp(today).floor('d')
    yesterday = today - pd.Timedelta(days=1)
    
    extraction_is_complete = data['at'].apply(lambda ts: ts < yesterday).any()
    return extraction_is_complete


def get_continue_extracting_reviews(data):
    continue_extracting_reviews = not(get_extraction_is_complete(data))
    return continue_extracting_reviews


def extract_next_batch(continuation_token):
    results, continuation_token = reviews(
        'com.tgc.sky.android',
        continuation_token=continuation_token
    )
    
    next_batch = pd.DataFrame(results)
    return next_batch, continuation_token


def extract_review_data():
    results, continuation_token = reviews(
        'com.tgc.sky.android',
        sort=Sort.NEWEST,
        count=200
    ) 
    review_data = pd.DataFrame(results)
    
    continue_extracting_reviews = get_continue_extracting_reviews(review_data)
    while(continue_extracting_reviews):
        next_batch, continuation_token = extract_next_batch(continuation_token)
        continue_extracting_reviews = get_continue_extracting_reviews(next_batch) 
        review_data = review_data.append(next_batch, ignore_index=True)
        time.sleep(1)
            
    return review_data

In [22]:
review_data = extract_review_data()

In [ ]:
review_data = pd.DataFrame(results)
review_data

filtered_review_data = review_data.drop(columns=['userImage', 'replyContent', 'repliedAt'])